## Kelley hu4d5-5 VH-R50A vdw step error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VH-R50A vdw step TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [1]:
import os
os.chdir("..")
from common_functions import *

### Ingesting original TI lambda production data 

In [2]:
os.chdir("./TI_data/VH-R50A")
dvdls_crg = pd.read_csv("R50A_crg_bound.csv")
dvdls_ub_crg = pd.read_csv("R50A_crg_unbound.csv")
geom_dvdls_vdw = pd.read_csv("R50A_vdw_bound.csv")
dvdls_ub_vdw = pd.read_csv("R50A_vdw_unbound.csv")


#### Initial TI estimate

In [4]:
dG_bd_crg = dvdls_crg.groupby("Lambda").mean()["weight_dvdl"].sum()
dG_ubd_crg = dvdls_ub_crg.groupby("Lambda").mean()["weight_dvdl"].sum()
ddG_crg = dG_bd_crg - dG_ubd_crg

dG_bd_vdw = geom_dvdls_vdw.groupby("Lambda").mean()["weight_dvdl"].sum()
dG_ubd_vdw = dvdls_ub_vdw.groupby("Lambda").mean()["weight_dvdl"].sum()
ddG_vdw = dG_bd_vdw - dG_ubd_vdw

empirical_value = 4.58

print("Original ddG (crg step): ")
print(f"{round(ddG_crg, 4)} kcal/mol")

print("Original ddG (vdw step): ")
print(f"{round(ddG_vdw, 4)} kcal/mol")

print("Original total ddG: ")
print(f"{round(ddG_crg + ddG_vdw, 4)} kcal/mol")

print()
print("Empirical value: ")
print(f"{empirical_value} kcal/mol")

Original ddG (crg step): 
0.9958 kcal/mol
Original ddG (vdw step): 
1.9825 kcal/mol
Original total ddG: 
2.9783 kcal/mol

Empirical value: 
4.58 kcal/mol


### Random forest model for vdw step

#### Splitting data into independent/dependent variable sets, removing correlated variables

In [5]:
X = geom_dvdls_vdw.drop(
    ['weight_dvdl', 'dvdl', 'Run', 'Lambda', '#Frame', "H384_chi2"
    ], axis=1)

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns
Y = geom_dvdls_vdw["weight_dvdl"]


In [6]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. Therefore, corrections will only be applied to the charging step for hu4D5-5 VH-R50A.

In [8]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    # ('pre_select', kbest),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls_vdw["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.5603
Training r2 std dev: 
0.0004
Avg. test r2: 
0.4173
Testing r2 std dev: 
0.0045


,Mean,Median
Y406_chi2,0.106937,0.106908
Y401_chi2,0.106881,0.110941
R408_chi4,0.103651,0.106962
H384_chi1,0.095558,0.095712
T407_chi1,0.093934,0.097142
Y382_chi1,0.092906,0.092340
V397_chi1,0.059090,0.059067
Y382_chi2,0.057230,0.057130
R408_chi3,0.053406,0.051437
R408_chi2,0.041922,0.041543
